In [1]:
import pandas as pd
import numpy as np
from joblib import dump, load

Increase regularization for all models
also try not using smote

In [2]:
X_train_sc = pd.read_csv('X_train_sc.csv')
X_val_sc = pd.read_csv('X_val_sc.csv')
y_train_enc = np.ravel(pd.read_csv('y_train_enc.csv'))
y_val_enc = np.ravel(pd.read_csv('y_val_enc.csv'))

#make train and val customer id frames and drop from data
X_train_id = X_train_sc['customerid']
X_train_sc.drop('customerid',axis=1,inplace=True)
X_val_id = X_val_sc['customerid']
X_val_sc.drop('customerid',axis=1,inplace=True)


In [3]:
from sklearn.model_selection import GridSearchCV


In [4]:
# from imblearn.over_sampling import SMOTE
# sm = SMOTE()
# X_train_sc, y_train_enc = sm.fit_resample(X_train_sc,y_train_enc)

In [5]:
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import cross_val_score
from sklearn.metrics import classification_report



In [6]:
def modeler(est,X_train,y_train,X_val,y_val):
    
    est.fit(X_train,y_train)
    
    
    train_acc_score = np.mean(cross_val_score(est,X_train,y_train,scoring='precision',cv=7,n_jobs=-1))
    val_acc_score = np.mean(cross_val_score(est,X_val,y_val,scoring='precision',cv=7,n_jobs=4))
    diff = abs(train_acc_score - val_acc_score)
    
    y_val_pred = est.predict(X_val)
    
    
    print(f'train precision: {round(train_acc_score,5)}')
    print('-'*40)
    print(f'val precision: {round(val_acc_score,5)}')
    print('-'*40)
    print(f'difference: {round(diff,7)}')
    print('-'*40)
    print(f'val report: ')
    print(classification_report(y_val,y_val_pred))
    print('-'*40)
    return est

In [7]:
lr = LogisticRegression(random_state=1,solver='liblinear',C=0.01,tol=0.1)

modeler(lr,X_train_sc,y_train_enc,X_val_sc,y_val_enc)
dump(lr,'models/1stlr.joblib')

train precision: 0.67713
----------------------------------------
val precision: 0.67921
----------------------------------------
difference: 0.0020796
----------------------------------------
val report: 
              precision    recall  f1-score   support

           0       0.85      0.91      0.88       971
           1       0.67      0.55      0.60       350

    accuracy                           0.81      1321
   macro avg       0.76      0.73      0.74      1321
weighted avg       0.80      0.81      0.80      1321

----------------------------------------


['models/1stlr.joblib']

This one is less accurate, but scores are still close. Might be good for bagging.

In [8]:

lr2 = LogisticRegression(random_state=1,solver='liblinear',C=0.01,tol=0.005)
modeler(lr2,X_train_sc,y_train_enc,X_val_sc,y_val_enc)

dump(lr2,'models/2ndlr.joblib')

train precision: 0.67408
----------------------------------------
val precision: 0.67954
----------------------------------------
difference: 0.005461
----------------------------------------
val report: 
              precision    recall  f1-score   support

           0       0.85      0.91      0.88       971
           1       0.68      0.55      0.61       350

    accuracy                           0.81      1321
   macro avg       0.76      0.73      0.74      1321
weighted avg       0.80      0.81      0.80      1321

----------------------------------------


['models/2ndlr.joblib']

In [9]:
from sklearn.linear_model import RidgeClassifier

ridge = RidgeClassifier(random_state=1)


In [10]:
param_grid = {'alpha':[1000], 'solver':['saga']}
grid = GridSearchCV(ridge,param_grid,n_jobs=4,scoring='precision')
modeler(grid,X_train_sc,y_train_enc,X_val_sc,y_val_enc)

ridge_best = grid.best_estimator_

dump(ridge_best,'models/ridge.joblib')

train precision: 0.71249
----------------------------------------
val precision: 0.76559
----------------------------------------
difference: 0.0531053
----------------------------------------
val report: 
              precision    recall  f1-score   support

           0       0.82      0.93      0.87       971
           1       0.69      0.45      0.54       350

    accuracy                           0.80      1321
   macro avg       0.76      0.69      0.71      1321
weighted avg       0.79      0.80      0.79      1321

----------------------------------------


['models/ridge.joblib']

In [11]:
grid.best_params_

{'alpha': 1000, 'solver': 'saga'}

In [12]:
lr3 = LogisticRegression(random_state=1)
param_grid = {
    'penalty':['l1'],
    'C':[0.01],
    'solver':['saga']}


grid = GridSearchCV(lr3,param_grid,n_jobs=4,scoring='precision')
modeler(grid,X_train_sc,y_train_enc,X_val_sc,y_val_enc)
lr3_best = grid.best_estimator_

dump(lr3_best,'models/3rdlr.joblib')

train precision: 0.74518
----------------------------------------
val precision: 0.0
----------------------------------------
difference: 0.7451807
----------------------------------------
val report: 
              precision    recall  f1-score   support

           0       0.82      0.94      0.87       971
           1       0.70      0.42      0.52       350

    accuracy                           0.80      1321
   macro avg       0.76      0.68      0.70      1321
weighted avg       0.79      0.80      0.78      1321

----------------------------------------


['models/3rdlr.joblib']

In [13]:
grid.best_params_

{'C': 0.01, 'penalty': 'l1', 'solver': 'saga'}

In [14]:
from sklearn.linear_model import SGDClassifier

sgd = SGDClassifier(random_state=1)

param_grid = {'loss':['modified_huber'],
             'penalty':['elasticnet'],
             'alpha':[1.5],
              'l1_ratio':[0.05],
             'learning_rate':['adaptive'],
             'eta0':[1]}
grid = GridSearchCV(sgd,param_grid,n_jobs=4,scoring='precision')
modeler(grid,X_train_sc,y_train_enc,X_val_sc,y_val_enc)
sgd_best = grid.best_estimator_

dump(sgd_best,'models/sgd.joblib')

train precision: 0.80703
----------------------------------------
val precision: 0.77804
----------------------------------------
difference: 0.0289827
----------------------------------------
val report: 
              precision    recall  f1-score   support

           0       0.78      0.98      0.86       971
           1       0.77      0.21      0.34       350

    accuracy                           0.78      1321
   macro avg       0.77      0.60      0.60      1321
weighted avg       0.77      0.78      0.72      1321

----------------------------------------


['models/sgd.joblib']

In [15]:
grid.best_params_

{'alpha': 1.5,
 'eta0': 1,
 'l1_ratio': 0.05,
 'learning_rate': 'adaptive',
 'loss': 'modified_huber',
 'penalty': 'elasticnet'}

In [20]:
from sklearn.svm import SVC

svc = SVC(random_state=1,probability=True)

param_grid = {'kernel':['rbf'],
             'C':[1],
             'gamma':['scale']
             }
grid = GridSearchCV(svc,param_grid,n_jobs=4,scoring='precision')
modeler(grid,X_train_sc,y_train_enc,X_val_sc,y_val_enc)
svc_best = grid.best_estimator_

dump(svc_best,'models/svc.joblib')

train precision: 0.6887
----------------------------------------
val precision: 0.65847
----------------------------------------
difference: 0.0302353
----------------------------------------
val report: 
              precision    recall  f1-score   support

           0       0.83      0.92      0.88       971
           1       0.69      0.49      0.57       350

    accuracy                           0.81      1321
   macro avg       0.76      0.71      0.72      1321
weighted avg       0.80      0.81      0.80      1321

----------------------------------------


['models/svc.joblib']

In [21]:
grid.best_params_

{'C': 1, 'gamma': 'scale', 'kernel': 'rbf'}

In [17]:
from sklearn.neighbors import KNeighborsClassifier

knn = KNeighborsClassifier()

param_grid = {'n_neighbors':[5],
             'weights':['uniform'],
             'algorithm':['auto'],
             'leaf_size':[90],
             'p':[3]}
grid = GridSearchCV(knn,param_grid,n_jobs=4,scoring='precision')
modeler(grid,X_train_sc,y_train_enc,X_val_sc,y_val_enc)
knn_best = grid.best_estimator_

dump(knn_best,'models/knn.joblisb')

train precision: 0.56244
----------------------------------------
val precision: 0.5367
----------------------------------------
difference: 0.0257406
----------------------------------------
val report: 
              precision    recall  f1-score   support

           0       0.83      0.87      0.85       971
           1       0.59      0.52      0.55       350

    accuracy                           0.78      1321
   macro avg       0.71      0.70      0.70      1321
weighted avg       0.77      0.78      0.77      1321

----------------------------------------


['models/knn.joblisb']

In [18]:
grid.best_params_

{'algorithm': 'auto',
 'leaf_size': 90,
 'n_neighbors': 5,
 'p': 3,
 'weights': 'uniform'}

In [19]:
# from sklearn.gaussian_process import GaussianProcessClassifier

# gp = GaussianProcessClassifier()

# param_grid = {'n_restarts_optimizer':[0]}
# grid = GridSearchCV(gp,param_grid,n_jobs=-1,scoring='accuracy')
# modeler(grid,X_train_sc,y_train_enc,X_val_sc,y_val_enc)
# gp_best = grid.best_estimator_


In [24]:
from sklearn.naive_bayes import GaussianNB

nb = GaussianNB()

param_grid = {'var_smoothing':[2]}

grid = GridSearchCV(nb,param_grid,n_jobs=4,scoring='precision')
modeler(grid,X_train_sc,y_train_enc,X_val_sc,y_val_enc)
nb_best = grid.best_estimator_

dump(nb_best,'models/nb.joblib')

train precision: 0.60452
----------------------------------------
val precision: 0.60108
----------------------------------------
difference: 0.0034369
----------------------------------------
val report: 
              precision    recall  f1-score   support

           0       0.85      0.86      0.85       971
           1       0.59      0.57      0.58       350

    accuracy                           0.78      1321
   macro avg       0.72      0.71      0.72      1321
weighted avg       0.78      0.78      0.78      1321

----------------------------------------


['models/nb.joblib']

In [25]:
grid.best_params_

{'var_smoothing': 2}

In [32]:
from sklearn.tree import DecisionTreeClassifier

tree = DecisionTreeClassifier(random_state=1)

param_grid = {'criterion':['entropy'],
             'splitter':['random'],
             'max_depth':[2],
             'max_features':['auto']}

grid = GridSearchCV(tree,param_grid,n_jobs=4,scoring='precision')
modeler(grid,X_train_sc,y_train_enc,X_val_sc,y_val_enc)
tree_best = grid.best_estimator_

dump(tree_best,'models/tree.joblib')

train precision: 0.766
----------------------------------------
val precision: 0.45175
----------------------------------------
difference: 0.3142461
----------------------------------------
val report: 
              precision    recall  f1-score   support

           0       0.75      0.98      0.85       971
           1       0.69      0.10      0.17       350

    accuracy                           0.75      1321
   macro avg       0.72      0.54      0.51      1321
weighted avg       0.73      0.75      0.67      1321

----------------------------------------


['models/tree.joblib']

In [33]:
grid.best_params_

{'criterion': 'entropy',
 'max_depth': 2,
 'max_features': 'auto',
 'splitter': 'random'}

In [36]:
from sklearn.ensemble import RandomForestClassifier

rfr = RandomForestClassifier(random_state=1)

param_grid = {'n_estimators':[100],
             'criterion':['gini'],
             'max_depth':[2],
             'max_features':['auto'],
             'oob_score':[True]}

grid = GridSearchCV(rfr,param_grid,n_jobs=4,scoring='precision')
modeler(grid,X_train_sc,y_train_enc,X_val_sc,y_val_enc)
rfr_best = grid.best_estimator_

dump(rfr_best,'models/rfr.joblib')

train precision: 0.75988
----------------------------------------
val precision: 0.65372
----------------------------------------
difference: 0.106158
----------------------------------------
val report: 
              precision    recall  f1-score   support

           0       0.79      0.96      0.86       971
           1       0.71      0.28      0.40       350

    accuracy                           0.78      1321
   macro avg       0.75      0.62      0.63      1321
weighted avg       0.77      0.78      0.74      1321

----------------------------------------


['models/rfr.joblib']

In [37]:
grid.best_params_

{'criterion': 'gini',
 'max_depth': 2,
 'max_features': 'auto',
 'n_estimators': 100,
 'oob_score': True}

In [38]:
from sklearn.ensemble import ExtraTreesClassifier

etree = ExtraTreesClassifier(random_state=1)
param_grid = {'n_estimators':[100],
             'criterion':['entropy'],
             'max_depth':[2],
             'max_features':['sqrt']}

grid = GridSearchCV(etree,param_grid,n_jobs=4,scoring='precision')
modeler(grid,X_train_sc,y_train_enc,X_val_sc,y_val_enc)
etree_best = grid.best_estimator_



dump(etree_best,'models/etree.joblib')

train precision: 0.89184
----------------------------------------
val precision: 0.76778
----------------------------------------
difference: 0.1240517
----------------------------------------
val report: 
              precision    recall  f1-score   support

           0       0.75      0.99      0.86       971
           1       0.86      0.09      0.16       350

    accuracy                           0.75      1321
   macro avg       0.80      0.54      0.51      1321
weighted avg       0.78      0.75      0.67      1321

----------------------------------------


['models/etree.joblib']

In [39]:
grid.best_params_

{'criterion': 'entropy',
 'max_depth': 2,
 'max_features': 'sqrt',
 'n_estimators': 100}

In [44]:
import xgboost as xgb


boost_model = xgb.XGBClassifier(random_state=1,verbosity=3,tree_method='gpu_hist')

param_grid = {'sampling_method':['gradient_based'],
             'colsample_bytree':[0.8],
             'grow_policy':[0.03,0.1],
             'gamma':[0.1,0.5]}

grid = GridSearchCV(boost_model,param_grid,n_jobs=4,scoring='precision')

xgb = modeler(grid,X_train_sc,y_train_enc,X_val_sc,y_val_enc)

dump(xgb,'models/xgb.joblib')

[15:01:12] ======== Monitor: Learner ========
[15:01:12] Configure: 0.000547s, 1 calls @ 547us

[15:01:12] ======== Monitor: GBTree ========
[15:01:12] ======== Monitor: TreePruner ========
[15:01:14] DEBUG: C:\Users\Administrator\workspace\xgboost-win64_release_1.2.0\src\gbm\gbtree.cc:147: Using tree method: 2
[15:01:14] INFO: C:\Users\Administrator\workspace\xgboost-win64_release_1.2.0\src\tree\updater_prune.cc:101: tree pruning end, 6 extra nodes, 0 pruned nodes, max_depth=2
[15:01:14] INFO: C:\Users\Administrator\workspace\xgboost-win64_release_1.2.0\src\tree\updater_prune.cc:101: tree pruning end, 6 extra nodes, 0 pruned nodes, max_depth=2
[15:01:14] INFO: C:\Users\Administrator\workspace\xgboost-win64_release_1.2.0\src\tree\updater_prune.cc:101: tree pruning end, 6 extra nodes, 0 pruned nodes, max_depth=2
[15:01:14] INFO: C:\Users\Administrator\workspace\xgboost-win64_release_1.2.0\src\tree\updater_prune.cc:101: tree pruning end, 6 extra nodes, 0 pruned nodes, max_depth=2
[15:01:

TerminatedWorkerError: A worker process managed by the executor was unexpectedly terminated. This could be caused by a segmentation fault while calling the function or by an excessive memory usage causing the Operating System to kill the worker.


In [ ]:
grid.best_params_

In [36]:
from sklearn.ensemble import AdaBoostClassifier

adatree = DecisionTreeClassifier(max_depth=1,criterion='gini',max_features='sqrt',random_state=1)

ada = AdaBoostClassifier(base_estimator=adatree,n_estimators=500,learning_rate=0.1,random_state=1)

ada = modeler(ada,X_train_sc,y_train_enc,X_val_sc,y_val_enc)

dump(ada,'models/ada.joblib')

train precision: 0.77052
----------------------------------------
val precision: 0.64999
----------------------------------------
difference: 0.1205249
----------------------------------------
val report: 
              precision    recall  f1-score   support

           0       0.90      0.74      0.81       971
           1       0.52      0.78      0.62       350

    accuracy                           0.75      1321
   macro avg       0.71      0.76      0.72      1321
weighted avg       0.80      0.75      0.76      1321

----------------------------------------


['ada.joblib']

In [37]:
from sklearn.ensemble import StackingClassifier

best_estimators = [('ridge',ridge_best),
                   ('lr',lr),
                   ('sgd',sgd_best),
                   ('svc',svc_best),
                   ('knn',knn_best),
                   ('nb',nb_best),
                   ('tree',tree_best),
                   ('rfr',rfr_best),
                   ('xgb',boost_model),
                   ('ada',ada),
                  ('etree',etree)]

stack = StackingClassifier(estimators=best_estimators,n_jobs=4)

stack = modeler(stack,X_train_sc,y_train_enc,X_val_sc,y_val_enc)

dump(stack,'models/stack.joblib')

train precision: 0.83879
----------------------------------------
val precision: 0.67361
----------------------------------------
difference: 0.165176
----------------------------------------
val report: 
              precision    recall  f1-score   support

           0       0.85      0.83      0.84       971
           1       0.56      0.60      0.58       350

    accuracy                           0.77      1321
   macro avg       0.71      0.72      0.71      1321
weighted avg       0.78      0.77      0.77      1321

----------------------------------------


['stack.joblib']

array([[0.9865013 , 0.0134987 ],
       [0.86873178, 0.13126822],
       [0.85128411, 0.14871589],
       ...,
       [0.22850614, 0.77149386],
       [0.99349563, 0.00650437],
       [0.13091009, 0.86908991]])